# Custom Keras Generator
This notebook outlines how to build custom Keras generators. These are used to feed in more complex data into models with potentially complex processing, such as image augmentation. It also allows for multiple datasets to be fed into the model for training and validation.

In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow

# Load entire dataset
X = np.linspace(0,100, 10000)
y = np.sin(X)

In [28]:
def data_generator(X,y, batch_size = 64):
    
    """
    Overview
    --------
    Simple Keras data generating function.
    
    
    Inputs
    ------
    X (numpy.ndarray): NumPy array of training data, with number
        of samples in the first dimension.
    y (numpy.ndarray): NumPy array of target data, with number
        of samples in the first dimension.
    batch_size (int, optional): Batch size.
    
    
    Returns
    -------
    Subset of X and y of size batch_size.
    """
    
    X_length = len(X)
    
    while True:
        # Select files (paths/indices) for the batch
        batch_idx = np.random.randint(0, X_length, (batch_size))
        
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array(X[batch_idx])
        batch_y = np.array(y[batch_idx])
        
        yield( batch_x, batch_y )

In [27]:
# Design model
model = Sequential()
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error')

In [25]:
model.fit_generator(image_generator(X, y, 32), steps_per_epoch = len(X)//64, epochs = 10)

Epoch 1/10
156/156 [==============================] - 2s 13ms/step - loss: 18.6917
Epoch 2/10
156/156 [==============================] - 2s 13ms/step - loss: 0.5521
Epoch 3/10
156/156 [==============================] - 2s 12ms/step - loss: 0.5546
Epoch 4/10
156/156 [==============================] - 2s 12ms/step - loss: 0.5482
Epoch 5/10
156/156 [==============================] - 2s 12ms/step - loss: 0.5512
Epoch 6/10
156/156 [==============================] - 2s 12ms/step - loss: 0.5452
Epoch 7/10
156/156 [==============================] - 2s 13ms/step - loss: 0.5406
Epoch 8/10
 87/156 [===============>..............] - ETA: 0s - loss: 0.5645

KeyboardInterrupt: 

In [12]:
100//3

33